<a href="https://colab.research.google.com/github/zemarchezi/NASASpaceApps/blob/main/Read_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install cdflib
!pip install pyspedas
!pip install matplotlib==3.1.2

In [ ]:
import glob
import cdflib 
import sys
import os
import requests
import pytz
import datetime
import pandas as pd
from bs4 import BeautifulSoup as bs
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

In [ ]:
DATASET_PATH = 'dataset/'

In [ ]:

def download_bwt_file(days, month, year='2022', data='wind', n=10):
    def get_links(days, month, year, url, filename):
        url = os.path.join(url, str(year))
        links = []
        for day in days:
            timestamp = f'{year:04d}{month:02d}{day:02d}'
            searchFilename = f"(" + filename.format(timestamp) + "_\w{3}\.cdf)"
            print(searchFilename)

            content = requests.get(url).content

            fnameS = re.search(searchFilename, str(content))
            print(fnameS)
            if fnameS:
                fname = fnameS.group()

            else:
                fname = "NoFilename"
                
            # f = filename.format(timestamp)
            link = os.path.join(url, fname)
            links.append(link)
        return links
    

    if data == 'dscovr':
        url = 'https://cdaweb.gsfc.nasa.gov/pub/data/dscovr/h0/mag/'
        filename = 'dscovr_h0_mag_{}'
    elif data == 'dscovr_ion':
        url = 'https://cdaweb.gsfc.nasa.gov/pub/data/dscovr/h1/faraday_cup/'
        filename = "dscovr_h1_fc_{}"
    elif data == 'wind':
        url = 'https://cdaweb.gsfc.nasa.gov/pub/data/wind/mfi/mfi_h2}/'
        filename = 'wi_h2_mfi_{}'
    elif data == 'wind_ion':
        url = 'https://cdaweb.gsfc.nasa.gov/pub/data/wind/swe/swe_h1/'
        filename = 'wi_h1_swe_{}'
    else:
        raise Exception('Conjunto de dados desconhecido')
    content = requests.get(url).content
    DATA_FOLDER = 'dataset/'
    if not os.path.exists(os.path.join(DATA_FOLDER, data)):
        os.makedirs(os.path.join(DATA_FOLDER, data))
    downloaded = 0
    for link in tqdm(get_links(days=days, month=month, 
                               year=year, url=url, filename=filename)):
        filename = os.path.basename(link)
        if downloaded == n:
            break
        print (link)
        response = requests.get(link)
        with open(f'{DATA_FOLDER}{data}/{filename}', 'wb') as f:
            f.write(response.content)
        downloaded += 1

   

def load_cdf_files(data='wind'):
    if data == 'dscovr':
        base = 'dataset/dscovr/'
    elif data == 'wind':
        base = 'dataset/wind/'
    elif data == 'wind_ion':
        base = 'dataset/wind_ion/'
    else:
        raise Exception('Conjunto de dados desconhecido')
    files = glob.glob(os.path.join(base, '*.cdf'))
    for f in files:
        yield cdflib.cdf_to_xarray(f)

In [ ]:
download_bwt_file(days=range(1, 11), month=1, year=2018, data='wind')

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
download_bwt_file(days=range(1, 11), month=1, year=2018, data='dscovr')

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
download_bwt_file(days=range(1, 11), month=1, year=2018, data='wind_ion')

  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

## Leitura de dados

In [ ]:
dscovr_ds = load_cdf_files(data='dscovr')
wind_ds = load_cdf_files(data='wind')
wind_ion_ds = load_cdf_files(data='wind_ion')

In [ ]:
def export_wind_df(ds, n_examples=10000):
  idx_file = 0
  for wind in wind_ds:
    df = pd.DataFrame(wind['BGSE'][:n_examples].values, columns=["Bx_GSM", "Gy_GSM", "Bz_GSM"])
    df['epoch'] = wind['Epoch'][:n_examples].values
    df.to_parquet(f'wind_{idx_file}.parquet')
    idx_file +=1

def export_dscovr_df(ds, n_examples=10000):
  idx_file = 0
  for dscovr in dscovr_ds:
    df = pd.DataFrame(dscovr['B1GSE'][:n_examples].values, columns=["Bx_GSM", "Gy_GSM", "Bz_GSM"])
    df['epoch'] = dscovr['Epoch1'][:n_examples].values
    df.to_parquet(f'dscovr_{idx_file}.parquet')
    idx_file +=1

In [ ]:
export_wind_df(wind_ds)

OSError: ignored

In [ ]:
export_dscovr_df(dscovr_ds)

In [ ]:
next(dscovr_ds)

<xarray.Dataset>
Dimensions:       (Epoch1: 86400, unit_time: 3, dim0: 3, dim1: 8)
Coordinates:
  * Epoch1        (Epoch1) float64 6.368e+13 6.368e+13 ... 6.368e+13 6.368e+13
  * unit_time     (unit_time) <U4 'year' 'day ' 'msec'
    label_time    (unit_time) <U27 'Year                       ' ... 'Elapsed...
Dimensions without coordinates: dim0, dim1
Data variables: (12/17)
    Time1_PB5     (Epoch1, unit_time) int32 2018 2 500 2018 ... 2018 2 86399500
    NUM1_PTS      (Epoch1) int32 50 50 50 50 50 50 50 ... 50 50 50 50 49 50 50
    B1F1          (Epoch1) float32 5.225 5.267 5.26 5.258 ... 3.49 3.505 3.483
    B1SDF1        (Epoch1) float32 0.0402 0.05895 0.03825 ... 0.0525 0.04315
    B1GSE         (Epoch1, dim0) float32 -0.1574 4.578 -2.513 ... 2.444 1.722
    B1SDGSE       (Epoch1, dim0) float32 0.06367 0.05277 ... 0.03717 0.05089
    ...            ...
    SENS          (dim0, dim1) float32 -1e+31 0.008067 -1e+31 ... -1e+31 -1e+31
    format_time   (dim0) <U2 'I4' 'I3' 'I8'
    label_bgse    (dim0) <U8 'Bx (GSE)' 'By (GSE)' 'Bz (GSE)'
    label_bsdgse  (dim0) <U14 'Bx_SIGMA (GSE)' 'By_SIGMA (GSE)' 'Bz_SIGMA (GSE)'
    label_brtn    (dim0) <U8 'Br (RTN)' 'Bt (RTN)' 'Bn (RTN)'
    label_bsdrtn  (dim0) <U14 'Br_SIGMA (RTN)' 'Bt_SIGMA (RTN)' 'Bn_SIGMA (RTN)'
Attributes: (12/21)
    Project:                     DSCOVR>Deep Space Climate Observatory
    Discipline:                  Space Physics>Heliospheric Science
    Source_name:                 DSCOVR>Deep Space Climate Observatory
    Software_version:            V01.00
    Data_type:                   H0>1-sec Definitive Data
    Descriptor:                  MAG>Fluxgate Magnetometer
    ...                          ...
    PI_name:                     A. Koval
    PI_affiliation:              UMBC, NASA/GSFC
    Mission_group:               DSCOVR
    Instrument_type:             Magnetic Fields (space)
    TEXT_supplement_1:            
    Web_sites:

In [ ]:
next(wind_ds)

<xarray.Dataset>
Dimensions:      (Epoch: 936007, unit_time: 3, dim0: 3, Epoch1: 1440, dim1: 8)
Coordinates:
  * Epoch        (Epoch) float64 6.368e+13 6.368e+13 ... 6.368e+13 6.368e+13
  * Epoch1       (Epoch1) float64 6.368e+13 6.368e+13 ... 6.368e+13 6.368e+13
  * unit_time    (unit_time) <U4 'year' 'day ' 'msec'
    label_time   (unit_time) <U27 'Year                       ' ... 'Elapsed ...
Dimensions without coordinates: dim0, dim1
Data variables: (12/26)
    Time_PB5     (Epoch, unit_time) int32 2018 2 36 2018 ... 2018 2 86399987
    BF1          (Epoch) float32 4.764 4.76 4.747 4.744 ... 3.69 3.679 3.694
    BGSM         (Epoch, dim0) float32 0.6269 4.719 0.1661 ... 2.987 1.713
    BGSE         (Epoch, dim0) float32 0.6269 4.595 -1.09 ... -1.339 3.34 0.836
    RANGE        (Epoch) int32 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1
    SPC_MODE     (Epoch) int32 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1
    ...           ...
    PAYLD1_I     (Epoch1, dim0, dim0) float64 1.0 0.0 0.0 0.0 ... 0.0 0.0 -1.0
    FLAG1_I      (Epoch1) int32 5 5 5 5 5 5 5 5 5 5 5 ... 5 5 5 5 5 5 5 5 5 5 5
    format_time  (dim0) <U2 'I4' 'I3' 'I8'
    label_bgsm   (dim0) <U8 'Bx (GSM)' 'By (GSM)' 'Bz (GSM)'
    label_bgse   (dim0) <U8 'Bx (GSE)' 'By (GSE)' 'Bz (GSE)'
    cartesian    (dim0) <U11 'x-component' 'y-component' 'z-component'
Attributes: (12/28)
    Project:                     ISTP>International Solar-Terrestrial Physics
    Discipline:                  Space Physics>Heliospheric Science
    Source_name:                 WIND>Wind Interplanetary Plasma Laboratory
    Software_version:            V05.00
    Data_type:                   H2>High-resolution Definitive Data
    Descriptor:                  MFI>Magnetic Fields Investigation
    ...                          ...
    PI_name:                     A. Szabo
    PI_affiliation:              NASA/GSFC
    Mission_group:               Wind
    Instrument_type:             Magnetic Fields (space)
    TEXT_supplement_1:            
    Web_sites:                   http://wind.nasa.gov

In [ ]:
next(wind_ion_ds)

<xarray.Dataset>
Dimensions:                   (Epoch: 868)
Coordinates: (12/20)
  * Epoch                     (Epoch) float64 6.368e+13 6.368e+13 ... 6.368e+13
    Proton_sigmaV_nonlin      (Epoch) float32 12.65 10.49 11.62 ... 16.17 16.43
    Proton_sigmaVX_nonlin     (Epoch) float32 12.71 10.55 11.69 ... 16.2 16.46
    Proton_sigmaVY_nonlin     (Epoch) float32 1.469 1.21 1.178 ... 3.579 3.735
    Proton_sigmaVZ_nonlin     (Epoch) float32 4.455 4.082 4.653 ... 1.0 1.0 1.0
    Proton_sigmaW_nonlin      (Epoch) float32 1.899 1.553 1.662 ... 4.603 7.562
    ...                        ...
    Alpha_sigmaVZ_nonlin      (Epoch) float32 19.81 27.3 24.54 ... 1.0 2.695
    Alpha_sigmaW_nonlin       (Epoch) float32 1.899 1.553 1.662 ... 4.603 7.562
    Alpha_sigmaWperp_nonlin   (Epoch) float32 1e+05 1e+05 1e+05 ... 1e+05 1e+05
    Alpha_sigmaWpar_nonlin    (Epoch) float32 1e+05 1e+05 1e+05 ... 1e+05 1e+05
    Alpha_sigmaNa_nonlin      (Epoch) float32 0.01 0.01 0.01 ... 0.01 0.01 0.01
    sigmaPeak_doy             (Epoch) float64 4.424e-05 7.373e-05 ... 7.373e-05
Data variables: (12/41)
    fit_flag                  (Epoch) int8 10 10 10 10 10 10 ... 10 2 3 10 3 10
    year                      (Epoch) int8 -30 -30 -30 -30 ... -30 -30 -30 -30
    doy                       (Epoch) float32 1.001 1.002 1.003 ... 1.998 2.0
    Proton_V_nonlin           (Epoch) float32 399.9 401.3 401.5 ... 427.9 431.2
    Proton_VX_nonlin          (Epoch) float32 -397.8 -398.8 ... -427.1 -430.3
    Proton_VY_nonlin          (Epoch) float32 -10.56 -10.7 ... 25.07 26.72
    ...                        ...
    dev                       (Epoch) float32 0.54 0.2249 ... 0.03479 0.04344
    xgse                      (Epoch) float32 199.9 199.9 199.9 ... 200.5 200.5
    ygse                      (Epoch) float32 79.87 79.87 79.87 ... 81.88 81.88
    zgse                      (Epoch) float32 -8.929 -8.929 ... -8.739 -8.739
    ygsm                      (Epoch) float32 79.47 79.48 79.48 ... 81.26 81.27
    zgsm                      (Epoch) float32 11.94 11.91 11.89 ... 13.32 13.3
Attributes: (12/19)
    Project:                     WIND/SWE Faraday Cup (Proton - Alpha Anisotr...
    Source_name:                 Wind
    Discipline:                  ['Space Physics>Magnetospheric Science', 'Sp...
    Data_type:                   VS>Validated Summary Data
    Descriptor:                  SWE>Solar Wind Experiment
    File_naming_convention:      source_datatype_descriptor
    ...                          ...
    Logical_file_id:             wi_h1_swe_20180101_v01
    Logical_source_description:  Solar wind proton and alpha parameters, incl...
    Time_resolution:             92 seconds
    Generated_by:                Michael L. Stevens (mstevens@cfa.harvard.edu)
    Generation_date:             Mon Feb 11 12:35:45 2019
    MODS:                        data analysis package revised March, 2012.